# Data Deduplication on transactions from Big Dive 7

In [1]:
import pandas as pd
import recordlinkage as rl

## loading dataframe from csv

In [2]:
transactions = pd.read_csv('transactions.csv')
transactions.head()

,date,description,amount,currency,account_id,account_type,user_id,provider
0,01/01/2017,GENIUS SUPER GENIUS 2.0 COSTO FISSO MESE DI DI...,7.35,EUR,3752062391,account,1272326183,bank-01
1,01/01/2017,GENIUS ONE EDIZIONE 3 COSTO FISSO MESE DI DICE...,1.40,EUR,2698932672,account,4248636026,bank-01
2,2017-01-01,PAYPAL *UBER BV 35314369001 GBR,28.26,EUR,898876702,card,163469645,bank-03
3,01/01/2017,H3G RICARICA AUTOMATICA TREZZANO SUL,10.00,EUR,1617404857,credit_card,1244820035,bank-16
4,01/01/2017,GENIUS SUPER GENIUS 2.0 COSTO FISSO MESE DI DI...,7.35,EUR,1839361966,account,3843080328,bank-01


In [3]:
transactions.dtypes

date             object
description      object
amount          float64
currency         object
account_id        int64
account_type     object
user_id           int64
provider         object
dtype: object

In [4]:
transactions.describe()

,amount,account_id,user_id
count,2.554885e+06,2.554885e+06,2.554885e+06
mean,6.843908e+01,2.108939e+09,2.157017e+09
std,8.642829e+02,1.232327e+09,1.236343e+09
min,1.000000e-02,1.827720e+05,6.391700e+04
25%,5.450000e+00,1.050150e+09,1.098814e+09
50%,1.399000e+01,2.095528e+09,2.159213e+09
75%,3.900000e+01,3.172311e+09,3.228949e+09
max,3.952804e+05,4.294890e+09,4.294663e+09


In [5]:
transactions.head()

,date,description,amount,currency,account_id,account_type,user_id,provider
0,01/01/2017,GENIUS SUPER GENIUS 2.0 COSTO FISSO MESE DI DI...,7.35,EUR,3752062391,account,1272326183,bank-01
1,01/01/2017,GENIUS ONE EDIZIONE 3 COSTO FISSO MESE DI DICE...,1.40,EUR,2698932672,account,4248636026,bank-01
2,2017-01-01,PAYPAL *UBER BV 35314369001 GBR,28.26,EUR,898876702,card,163469645,bank-03
3,01/01/2017,H3G RICARICA AUTOMATICA TREZZANO SUL,10.00,EUR,1617404857,credit_card,1244820035,bank-16
4,01/01/2017,GENIUS SUPER GENIUS 2.0 COSTO FISSO MESE DI DI...,7.35,EUR,1839361966,account,3843080328,bank-01


In [6]:
transactions.currency.unique()

array(['EUR', 'GBP'], dtype=object)

In [7]:
transactions.account_type.unique()

array(['account', 'card', 'credit_card', 'checking', 'debit_card',
       'savings', 'investment', 'credit', 'loan', 'bonus', 'mortgage',
       'insurance'], dtype=object)

In [8]:
transactions.provider.unique()

array(['bank-01', 'bank-03', 'bank-16', 'bank-15', 'bank-08', 'bank-14',
       'bank-17', 'bank-13', 'bank-04', 'bank-06', 'bank-07', 'bank-11',
       'bank-12', 'bank-10', 'bank-09', 'bank-02', 'bank-05', 'bank-18',
       'bank-20', 'bank-19', 'bank-25', 'bank-22', 'bank-23', 'bank-24',
       'bank-21', 'bank-26', 'bank-27', 'bank-31', 'bank-28', 'bank-29',
       'bank-30', 'bank-33', 'bank-32', 'bank-34', 'bank-35', 'bank-36',
       'bank-37', 'bank-38', 'bank-39', 'bank-40', 'bank-42', 'bank-44',
       'bank-43', 'bank-41', 'bank-45', 'bank-46', 'bank-47', 'bank-48',
       'bank-49', 'bank-50', 'bank-51', 'bank-52', 'bank-53', 'bank-54',
       'bank-55', 'bank-56', 'bank-57', 'bank-58', 'bank-59', 'bank-60',
       'bank-62', 'bank-61', 'bank-63', 'bank-64', 'bank-65', 'bank-66',
       'bank-68', 'bank-67', 'bank-70', 'bank-69', 'bank-71', 'bank-72',
       'bank-73', 'bank-74', 'bank-75', 'bank-76', 'bank-77', 'bank-78',
       'bank-79', 'bank-80', 'bank-81', 'bank-82', 

In [9]:
transactions['date'] = pd.to_datetime(transactions['date'], dayfirst=True, cache=True)

In [10]:
transactions.head()

,date,description,amount,currency,account_id,account_type,user_id,provider
0,2017-01-01,GENIUS SUPER GENIUS 2.0 COSTO FISSO MESE DI DI...,7.35,EUR,3752062391,account,1272326183,bank-01
1,2017-01-01,GENIUS ONE EDIZIONE 3 COSTO FISSO MESE DI DICE...,1.40,EUR,2698932672,account,4248636026,bank-01
2,2017-01-01,PAYPAL *UBER BV 35314369001 GBR,28.26,EUR,898876702,card,163469645,bank-03
3,2017-01-01,H3G RICARICA AUTOMATICA TREZZANO SUL,10.00,EUR,1617404857,credit_card,1244820035,bank-16
4,2017-01-01,GENIUS SUPER GENIUS 2.0 COSTO FISSO MESE DI DI...,7.35,EUR,1839361966,account,3843080328,bank-01


In [11]:
len(transactions.groupby('amount').count())

53617

In [12]:
len(transactions.groupby('account_id').count())

27353

In [13]:
len(transactions.groupby('user_id').count())

13236

## indexing

In [14]:
# Returns all record pairs that agree on the given variables.
# This method is known as blocking. Blocking is an effective way 
# to make a subset of the record space (A * B).
indexer = rl.BlockIndex(on=['amount', 'account_id', 'date'])

In [15]:
pairs = indexer.index(transactions)

In [16]:
len(pairs)

297754

## comparing

In [17]:
cmp = rl.Compare()
cmp.exact('currency', 'currency', label='currency')
cmp.string('description', 'description', label='description')
features = cmp.compute(pairs, transactions)

In [18]:
features.head()

currency  description
5  76         1          1.0
   77         1          1.0
   81         1          1.0
76 77         1          1.0
   81         1          1.0

## matching

In [19]:
matches = features[features.sum(axis=1) > 1.9]

In [20]:
matches.head()

currency  description
5  76         1          1.0
   77         1          1.0
   81         1          1.0
76 77         1          1.0
   81         1          1.0

## verifying results

In [21]:
transactions.loc[[2553873, 2553874]]

,date,description,amount,currency,account_id,account_type,user_id,provider
2553873,2017-12-31,IZ O.A.TAXI & MIETWAGEN KONIGS WUSTER,56.5,EUR,1516715854,credit_card,2005235773,bank-16
2553874,2017-12-31,IZ *O.A.TAXI & MIETWAGEN KONIGS WUSTER,56.5,EUR,1516715854,credit_card,2005235773,bank-16


In [22]:
transactions.loc[[396, 424, 542]]

,date,description,amount,currency,account_id,account_type,user_id,provider
396,2017-01-02,Non-Sterling transaction fee,0.23,GBP,2886701504,checking,2181840998,bank-19
424,2017-01-02,Non-Sterling transaction fee,0.23,GBP,2886701504,checking,2181840998,bank-19
542,2017-01-02,Non-Sterling transaction fee,0.23,GBP,2886701504,checking,2181840998,bank-19
